In [28]:
import os
from werkzeug.utils import secure_filename
import numpy as np # linear algebra
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from tqdm import tqdm
import shutil 
import matplotlib.pyplot as plt
import time
import copy
from PIL import Image


In [29]:
#connect to google drive where dataset is model h5
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [30]:
!ls /content/gdrive/'My Drive'/dataset0011/

dataset0011.zip  model_0011_01.h5  test  train


In [31]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

ALLOWED_EXTENSIONS=set(['jpg', 'jpeg'])

In [32]:
def allowed_file(filename):
    return '.' in filename and filename.lower().rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

In [39]:
model = torch.load("../content/gdrive/My Drive/dataset0011/model_0011_01.h5")

UPLOAD_FOLDER = '../content/gdrive/My Drive/dataset0011/'
data_root = '../content/gdrive/My Drive/dataset0011/'
test_dir = '../content/gdrive/My Drive/dataset0011/'

In [44]:
def get_prediction():
     
    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])  
    test_dataset = ImageFolderWithPaths(test_dir, val_transforms)
    #test_dataset = torchvision.datasets.ImageFolder(test_dir, val_transforms)

    test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=2, shuffle=False, num_workers=0)
    model.eval()

    test_predictions = []
    test_img_paths = []
    for inputs, labels, paths in tqdm(test_dataloader):
        #inputs = inputs.to(device)
        #labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
        test_predictions.append(
            torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
        test_img_paths.extend(paths)
    
    test_predictions = np.concatenate(test_predictions)

    inputs, labels, paths = next(iter(test_dataloader))
    submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
    submission_df['label'] = submission_df['label'].map(lambda pred: 'People' if pred > 0.5 else 'NoPeople')
    submission_df['id'] = submission_df['id'].str.replace('../content/gdrive/My Drive/dataset0011/', '')
    submission_df['id'] = submission_df['id'].str.replace('.jpg', '')    
    return submission_df

In [45]:
res = get_prediction()
print(res)

100%|██████████| 32/32 [00:13<00:00,  2.46it/s]


                   id     label
0    test/IMG_9593_00  NoPeople
1    test/IMG_9594_11  NoPeople
2    test/IMG_9601_00  NoPeople
3    test/IMG_9607_11    People
4    test/IMG_9614_11    People
..                ...       ...
58  train/11/IMG_9622  NoPeople
59  train/11/IMG_9623  NoPeople
60  train/11/IMG_9624    People
61  train/11/IMG_9626  NoPeople
62  train/11/IMG_9629  NoPeople

[63 rows x 2 columns]


In [46]:
!pip freeze  > requirements.txt

In [47]:
!ls

gdrive	requirements.txt  sample_data


In [48]:
from google.colab import files

files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>